In [2]:
from utilities.feed import get_stock_info
import utilities.variables as variables 
import yfinance as yf
import pandas as pd
import importlib
import utilities.parser as parser
import utilities.esg_service as esg_service
importlib.reload(parser)
importlib.reload(esg_service)

<module 'utilities.esg_service' from '/Users/herbishtini/Documents/UNI/Master Thesis/sustainability_portfolio_optimisation/utilities/esg_service.py'>

In [6]:
# esg_service.generate_esg_data()
#esg_service.generate_esg_data_for_each_industry()

In [4]:
ticker = yf.Ticker('3462.T')
print(ticker.info)

{'address1': 'Nishishinjuku 8-5-1', 'address2': 'Shinjuku-ku', 'city': 'Tokyo', 'zip': '160-0023', 'country': 'Japan', 'phone': '81 3 3365 8767', 'website': 'https://www.nre-mf.co.jp', 'industry': 'REIT - Retail', 'industryKey': 'reit-retail', 'industryDisp': 'REIT - Retail', 'sector': 'Real Estate', 'sectorKey': 'real-estate', 'sectorDisp': 'Real Estate', 'longBusinessSummary': 'Nomura Real Estate Master Fund, Inc. (NMF) is a real estate investment corporation formed to own and invest primarily in office buildings, retail facilities, logistics and residential facilities. NMF is externally managed by a licensed asset management company, Nomura Real Estate Asset Management Co., Ltd. ("NREAM"). NREAM is a wholly-owned subsidiary of Nomura Real Estate Holdings, Inc. NMF was established on October 1, 2015 through the consolidation type merger involving the former Nomura Real Estate Master Fund, Inc. (hereinafter the "former NMF"), Nomura Real Estate Office Fund, Inc. (hereinafter "NOF") an

In [22]:
# List of stock symbols
symbols = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']

# Dictionary to store market capitalizations
market_caps = {}

# Fetch market capitalization for each symbol
for symbol in symbols:
    ticker = yf.Ticker(symbol)
    market_cap = ticker.info.get('marketCap', 'pegRatio')  # Use 'N/A' if marketCap is not available
    market_caps[symbol] = market_cap

# Print the market capitalizations
for symbol, market_cap in market_caps.items():
    print(f"{symbol}: {market_cap}")

AttributeError: 'list' object has no attribute 'upper'

In [ ]:
msft.info['marketCap']

In [ ]:
msft.capital_gains

In [ ]:
tickers = yf.Tickers('msft aapl goog')
tickers.tickers['msft'].info

In [ ]:
pd.DataFrame(hist)

In [ ]:
msft.news